In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out import create_dir
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.point_clouds.ae_templates import conv_architecture_ala_nips_17, default_train_params_ala_nips_17
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.in_out import load_crude_point_clouds, PointCloudDataSet

from tf_lab.in_out.basics import read_saved_epochs          
from tf_lab.data_sets.shape_net import pc_loader as snc_loader
from tf_lab.data_sets.shape_net import snc_category_to_synth_id

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
n_pc_points = 2048
do_training = True
first_time_running = True
load_pretrained_model = False
load_epoch = None
seed = 42

In [8]:
max_training_epochs = 300
loss = 'chamfer'
bneck_size = 64

class_name = raw_input('Give me the class type: ').lower()
syn_id = snc_category_to_synth_id()[class_name]

experiment_id = raw_input('Give me the experiment ID:').lower()

experiment_name = '_'.join([class_name, experiment_id, str(n_pc_points), 
                           'pts_bneck', str(bneck_size), loss
                           ])

print experiment_name

# top_data_dir = '/orions4-zfs/projects/optas/DATA/'
# train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/testing_ae_settings', experiment_name)
# create_dir(train_dir)

Give me the class type: chair
Give me the experiment ID:213
chair_213_2048_pts_bneck_64_chamfer


In [9]:
# Load complete Point-Clouds of single class
pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_points))
pclouds_path = osp.join(pclouds_path, syn_id)
file_names = [f for f in files_in_subdirs(pclouds_path, '.ply')]
pclouds, model_ids, syn_ids = load_crude_point_clouds(file_names=file_names, n_threads=50, loader=snc_loader)
assert(len(np.unique(syn_ids)) == 1)
print '%d files containing complete point clouds were found.' % (len(pclouds), )
train_data = PointCloudDataSet(pclouds, labels=model_ids)

6778 files containing complete point clouds were found.


In [2]:
arch_params = conv_architecture_ala_nips_17(n_pc_points)
train_params = default_train_params_ala_nips_17(n_pc_points)

NameError: name 'conv_architecture_ala_nips_17' is not defined

In [218]:
arch_params['encoder_args']['n_filters'] = 128

In [219]:
train_dir = '/orions4-zfs/projects/optas/DATA/OUT/icml/nn_models/ae_farming/ae_farm_chair_conv_arch_8_2048pts_chamfer/'

In [220]:
load_pretrained_model = True

In [224]:
if load_pretrained_model:
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    print conf
    if conf.train_dir != train_dir:
        conf.train_dir = train_dir
    conf.save(osp.join(train_dir, 'configuration'))
else:
    conf = Conf(
                n_input = [n_pc_points, 3],
                loss = loss,
                training_epochs = train_params['training_epochs'],
                batch_size = train_params['batch_size'],
                denoising = train_params['denoising'],
                learning_rate = train_params['learning_rate'],
                train_dir = train_dir,              
                loss_display_step = 1,
                saver_step = None,
                z_rotate = train_params['z_rotate'],
                encoder = arch_params['encoder'],
                decoder = arch_params['decoder'],                
                encoder_args = arch_params['encoder_args'],
                decoder_args = arch_params['decoder_args']               
               )
    
    conf.experiment_name = experiment_name
#     conf.save(osp.join(conf.train_dir, 'configuration'))

                    batch_size: 50
                 consistent_io: None
                         debug: False
                       decoder: decoder_with_fc_only
                  decoder_args: {'layer_sizes': [1024, 2048, 6144]}
                       encoder: encoder_with_convs_and_symmetry
                  encoder_args: {'filter_sizes': [40, 20, 10, 10], 'n_filters': [128, 128, 256, 128], 'strides': [1, 2, 2, 1]}
               experiment_name: ae_farm_chair_conv_arch_8_2048pts_chamfer
                 gauss_augment: None
                  is_denoising: False
               latent_vs_recon: 1.0
                 learning_rate: 0.0005
                          loss: chamfer
             loss_display_step: 1
                       n_input: [2048, 3]
                      n_output: [2048, 3]
                           n_z: None
             saver_max_to_keep: None
                    saver_step: 10
                     train_dir: /orions4-zfs/projects/optas/DATA/OUT/icml/nn_models/ae_

In [227]:
reset_tf_graph()
ae = PointNetAutoEncoder(conf.experiment_name, conf)

In [3]:
if load_pretrained_model:
    if load_epoch is None: # load last saved.
        saved_epochs = read_saved_epochs(conf.train_dir)
        load_epoch = saved_epochs[-1]        
    ae.restore_model(conf.train_dir, load_epoch, verbose=True)

NameError: name 'load_pretrained_model' is not defined

In [210]:
bind_new_method_to_instance(partial_fit, ae)

In [207]:
from tf_lab.point_clouds.convenience import reconstruct_pclouds

In [213]:
rec, loss = reconstruct_pclouds(ae, train_data.point_clouds, 300)
print loss

0.00180301837036


In [211]:
if do_training:
    buf_size = 1 # flush each line

    fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)    
    train_stats = ae.train(train_data, conf, log_file=fout)

    fout.close()

('Epoch:', '0011', 'training time (minutes)=', '0.1979', 'loss=', '0.002349562')
('Epoch:', '0012', 'training time (minutes)=', '0.1984', 'loss=', '0.002187133')
('Epoch:', '0013', 'training time (minutes)=', '0.1993', 'loss=', '0.002074259')
('Epoch:', '0014', 'training time (minutes)=', '0.2004', 'loss=', '0.001986993')
('Epoch:', '0015', 'training time (minutes)=', '0.2072', 'loss=', '0.001884495')
('Epoch:', '0016', 'training time (minutes)=', '0.2066', 'loss=', '0.001806534')


UnboundLocalError: local variable 'recon' referenced before assignment

In [212]:
ae.is_training()

False

In [194]:
g = ae.graph
for variable in g.get_collection('variables'):
    print variable.name, variable.get_shape()

ae_farm_chair_increasing_stride_2048pts_bneck_128_chamfer/epoch:0 ()
ae_farm_chair_increasing_stride_2048pts_bneck_128_chamfer/encoder_conv_layer_0/W:0 (40, 1, 3, 128)
ae_farm_chair_increasing_stride_2048pts_bneck_128_chamfer/encoder_conv_layer_0/b:0 (128,)
ae_farm_chair_increasing_stride_2048pts_bneck_128_chamfer/encoder_conv_layer_0_bnorm/beta:0 (128,)
ae_farm_chair_increasing_stride_2048pts_bneck_128_chamfer/encoder_conv_layer_0_bnorm/gamma:0 (128,)
ae_farm_chair_increasing_stride_2048pts_bneck_128_chamfer/encoder_conv_layer_0_bnorm/moving_mean:0 (128,)
ae_farm_chair_increasing_stride_2048pts_bneck_128_chamfer/encoder_conv_layer_0_bnorm/moving_variance:0 (128,)
ae_farm_chair_increasing_stride_2048pts_bneck_128_chamfer/encoder_conv_layer_0_bnorm/is_training:0 ()
ae_farm_chair_increasing_stride_2048pts_bneck_128_chamfer/encoder_conv_layer_1/W:0 (20, 1, 128, 128)
ae_farm_chair_increasing_stride_2048pts_bneck_128_chamfer/encoder_conv_layer_1/b:0 (128,)
ae_farm_chair_increasing_stride_20

In [205]:
from tflearn import get_training_mode, 
ae.sess.run(get_training_mode())

True

In [196]:
b_tensor_name = 'ae_farm_chair_increasing_stride_2048pts_bneck_128_chamfer/encoder_conv_layer_1_bnorm/moving_variance:0'
ae.sess.run(g.get_tensor_by_name(b_tensor_name))


array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.], dtype=float32)

15822464

In [ ]:
# Plot Some feed-reconstruction pairs.
feed = train_data.next_batch(1)[0]
rec = ae.reconstruct(feed)[0]
Point_Cloud(feed[0]).plot();
Point_Cloud(rec[0]).plot();

In [ ]:
# sys.path.append('/orions4-zfs/projects/optas/Git_Repos/external_tools/YellowFin/tuner_utils')
# from yellowfin import YFOptimizer
# ae.optimizer = YFOptimizer